# {{cookiecutter.model_name}}

LSTM-based neural network model for time series prediction.

In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import plotly.graph_objects as go
# User modules
import utils
import model as ml
from conversion import Converter

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device.type

## Loading a Dataset

Some useful data loaders and handlers:
* [pandas](https://pandas.pydata.org/docs/user_guide/io.html)

In [ ]:
dataset = utils.load_dataset()

Some useful plotters:
* [Matplotlib](https://matplotlib.org/)
* [Plotly](https://plotly.com/python/)
* [bokeh](https://docs.bokeh.org/en/latest/index.html)

In [ ]:
utils.plot(go.Scatter(x=dataset.index, y=dataset['column_name'].values.astype('float')))

## Preparing a Data

Some useful data preparation libs:
* [NumPy](https://numpy.org/)
* [SciPy](https://www.scipy.org/)
* [scikit-learn](https://scikit-learn.org/stable/)
* [statsmodels](https://www.statsmodels.org/stable/index.html)

In [ ]:
converter = Converter(feature_range=(-1, 1))

In [ ]:
TEST_DATA_SIZE: int = None

train_data, test_data = utils.split_train_test(
    dataset['column_name'].values.astype('float'),
    TEST_DATA_SIZE
)

In [ ]:
train_data_normalized = converter.forward(train_data)

In [ ]:
TRAIN_WINDOW_SIZE: int = None

train_inout_seq = utils.create_inout_sequences(train_data_normalized, TRAIN_WINDOW_SIZE)

## Model Creation

See:
* [Module](https://pytorch.org/docs/stable/nn.html#module)
* [Loss functions](https://pytorch.org/docs/stable/nn.html#loss-functions)
* [Optimizers](https://pytorch.org/docs/stable/optim.html#algorithms)

In [ ]:
model = ml.LSTM().to(device)
loss_function = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## Model Training

In [ ]:
epochs, loss = utils.train_model(model, device, train_inout_seq, loss_function, optimizer, epochs=150)

In [ ]:
utils.plot_loss(go.Scatter(x=epochs, y=loss))

## Predictions

In [ ]:
FUTURE_PREDICTIONS_SIZE: int = None

test_inputs = train_data_normalized[-TRAIN_WINDOW_SIZE:].tolist()

In [ ]:
model.eval()

predictions = ml.predict(
    model, device, test_inputs, future_predictions_size=FUTURE_PREDICTIONS_SIZE,
    train_window_size=TRAIN_WINDOW_SIZE
)

predictions = converter.backward(predictions)

In [ ]:
utils.plot(
    go.Scatter(x=dataset.index, y=dataset['column_name']),
    go.Scatter(x=dataset.index[-FUTURE_PREDICTIONS_SIZE:], y=predictions.flatten()),
)

## Saving a Model

See:
* [SAVING AND LOADING MODELS](https://pytorch.org/tutorials/beginner/saving_loading_models.html)
* [SAVING AND LOADING MODELS FOR INFERENCE IN PYTORCH](https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_models_for_inference.html)
* [TORCH.ONNX](https://pytorch.org/docs/stable/onnx.html)
* [ONNX](https://onnx.ai/)

In [ ]:
torch.save(model.state_dict(), 'model.pth')